In [ ]:
!pip install nltk dagshub pickle

ERROR: Could not find a version that satisfies the requirement pickle (from versions: none)
ERROR: No matching distribution found for pickle
You should consider upgrading via the '/opt/venv/bin/python -m pip install --upgrade pip' command.


In [ ]:
import pandas as pd
import re
from nltk.tokenize import RegexpTokenizer
import numpy as np
import matplotlib.pyplot as plt
import sys

from sklearn.metrics import *
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.ensemble import BaggingClassifier
from sklearn.multiclass import OneVsRestClassifier

import pickle
import dagshub

In [ ]:
def load_corpus(DATASET_PATH, CODE_COLUMN):
    df = pd.read_csv(DATASET_PATH, encoding='utf-8', comment='#', sep='\t')#, quoting=csv.QUOTE_NONE, error_bad_lines=False)#, sep=','
    corpus = df[CODE_COLUMN]
    test_size = 0.1
    test_rows = round(df.shape[0]*test_size)
    train_rows = df.shape[0] - test_rows
    train_corpus = df[CODE_COLUMN][0:test_rows]
    test_corpus = df[CODE_COLUMN][train_rows:]
    return df, corpus

In [ ]:
def tfidf_transform(corpus, params):
#     tfidf = TfidfVectorizer(min_df=5
#                             , max_df = 0.3
#                             , ngram_range = (1,2)
#                             , smooth_idf = True
#                            )
#     tfidf = TfidfVectorizer(params)
#     features = tfidf.fit_transform(corpus)
    vectorizer = TfidfVectorizer(ngram_range = (1,2), smooth_idf = True)
    features = vectorizer.fit_transform(corpus)
    # for_pred = tfidf.transform(test_corpus)
    return features

In [ ]:
def SVM_evaluate(df, features):
    X_train, X_test, y_train, y_test = train_test_split(features, df[TAG_TO_PREDICT], test_size=0.3)
    grid = {"C": [1, 10, 100]}
    cv = KFold(n_splits=5, shuffle=True, random_state=241)
    model = SVC(kernel="linear", random_state=241)
    gs = GridSearchCV(model, grid, scoring="accuracy", cv=cv, verbose=1, n_jobs=-1)
    gs.fit(X_train[:25000], y_train.ravel()[:25000])
    C = gs.best_params_.get('C')
    # model = SVC(C=C, kernel="linear", random_state=241)
    # model.fit(X_train, y_train.ravel())
    # a faster option:
    # 1: usage of BagginClassifier decreased the fitting time from 38 mins to 8
    n_estimators = 10
    clf = BaggingClassifier(SVC(C=C, kernel='linear', random_state=241), max_samples=1.0 / n_estimators, n_estimators=n_estimators)
    clf.fit(X_train, y_train.ravel())
    # y_pred = model.predict(X_test)
    y_pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    metrics = {'test_accuracy': accuracy
               , 'test_f1_score': f1}
    plot_confusion_matrix(model, X_test, y_test)
    return metrics

In [ ]:
def get_predictions(X, y, TAG_TO_PREDICT, MODEL_DIR):
    clf = pickle.load(open(MODEL_DIR, 'rb'))
    # result = loaded_model.score(X, y)
    y_pred = clf.predict(X)
    accuracy = clf.score(X, y)
    f1 = f1_score(y_pred, y)
    print(f'Mean Accuracy {round(accuracy*100, 2)}%')
    print(f'F1-score {round(f1*100, 2)}%')
    errors = y - y_pred
    plt.hist(errors)
    plot_precision_recall_curve(clf, X, y)
    plot_confusion_matrix(clf, X, y, values_format='d')
    def mean_confidence_interval(data, confidence=0.95):
        a = 1.0 * np.array(data)
        n = len(a)
        m, se = np.mean(a), scipy.stats.sem(a)
        h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
        return m, m-h, m+h
    conf_interval = mean_confidence_interval(errors, 0.95)
    print(conf_interval)
    metrics = {'test_accuracy': accuracy
               , 'test_f1_score': f1}
    return metrics
    print(result)

In [ ]:
if __name__ == '__main__':
    DATASET_PATH = '/home/jovyan/work/chunks_40_validate.csv'
    MODEL_DIR = '/home/jovyan/work/svm_regex.sav'
    TFIDF_DIR = '/home/jovyan/work/tfidf.pickle'
    CODE_COLUMN = 'code'
    TAG_TO_PREDICT = 'tag'
    df, corpus = load_corpus(DATASET_PATH, CODE_COLUMN)
    nrows = df.shape[0]
    print("loaded")
    tfidf_params = {'min_df': 5
             , 'max_df': 0.3
             , 'smooth_idf': True}
    data_meta = {'DATASET_PATH': DATASET_PATH
                ,'nrows': nrows
                ,'label': TAG_TO_PREDICT
                ,'model': 'SVM'}
    features = tfidf_transform(corpus, tfidf_params)
    print("tfidf-ed")
    with dagshub.dagshub_logger() as logger:
        # metrics = SVM_evaluate(df, features)
        metrics = get_predictions(features, df[TAG_TO_PREDICT], TAG_TO_PREDICT, MODEL_DIR)
        logger.log_hyperparams(data_meta)
        logger.log_hyperparams(params)
        logger.log_metrics(metrics)
    print("finished")

loaded
tfidf-ed


FileNotFoundError: [Errno 2] No such file or directory: '/home/jovyan/work/svm_regex.sav'